# Layoffs 2022: Data Cleaning (Part 1 of 2)

# Introduction

This SQL project is centered on the "Layoffs 2022" dataset sourced from Kaggle. The dataset offers a wealth of information regarding companies, industries, and pertinent details surrounding the layoffs that transpired in 2022. In order to ensure precise analysis and meaningful findings, we will execute a series of data cleaning procedures as detailed below.

## Project Overview

1. **Creating Staging Table:**

   We begin by creating a staging table, `layoffs_staging`, to work with the raw data. This ensures preservation of the original dataset in case of any unexpected changes.

2. **Data Cleaning Steps:**
   - **Checking for Duplicates:**
     - Identifying and removing duplicate entries based on company, location, industry, total layoffs, and other relevant columns.
     - Utilizing window functions to identify duplicate rows and deleting them from the staging table.

   - **Standardizing Data:**
     - Standardizing variations of industry names into a single consistent value.
     - Standardizing country names by removing trailing characters for uniformity.

   - **Correcting Date Format:**
     - Converting the 'date' column to a proper date format and updating the data type to DATE for consistency.

   - **Handling Null Values:**
     - Reviewing and deciding to retain NULL values in columns for ease of calculations during Exploratory Data Analysis (EDA).

   - **Removing Unnecessary Data:**
     - Removing rows with NULL values in critical columns, ensuring the dataset contains only relevant and usable data.
     - Dropping the temporary column created for duplicate identification.

By meticulously cleaning and standardizing the data, we aim to create a refined dataset ready for in-depth analysis. These steps ensure that our subsequent analyses and visualizations are based on accurate, consistent, and reliable information from the "Layoffs 2022" dataset.

## Dataset Overview

This dataset, sourced from [Kaggle](https://www.kaggle.com/datasets/swaptr/layoffs-2022), tracks tech layoffs reported across various platforms:
- Bloomberg
- San Francisco Business Times
- TechCrunch
- The New York Times

The data spans from the declaration of COVID-19 as a pandemic on March 11, 2020, to the present (April 20, 2024). Certain data, such as sources, lists of laid-off employees, and addition dates, have been omitted but are available on Layoffs.fyi.

### Columns Descriptions


| Column Name         | Description                                                                                                 |
|---------------------|-------------------------------------------------------------------------------------------------------------|
| **company**         | Name of the company.                                                                   |
| **location**        | Location of the company headquarters.                                                                        |
| **industry**        | Industry of the company.                                                                                    |
| **total_laid_off**  | Number of employees laid off.                                                                              |
| **percentage_laid_off** | Percentage of employees laid off.                                                                        |
| **date**            | Date of layoff.                                                                                             |
| **stage**           | Stage of funding.                                                                                           |
| **country**         | Country.                                                                                                    |
| **funds_raised**    | Funds raised by the company (in Millions $).                                                               |

# Connect to Database

We will use the SQLite database in this project to easily work with SQL on Google Colab.

In [ ]:
import sqlite3

# Connect to an SQLite database; use ':memory:' for an in-memory database
conn = sqlite3.connect('covid_data.db')

In [ ]:
%%capture
# Install ipython-sql
!pip install ipython-sql

In [ ]:
# Load the SQL extension
%load_ext sql

# Create a SQLite database
%sql sqlite:///covid_data.db

# Load the Dataset

We will import the dataset from CSV files into the SQLite database we've created.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Datasets/layoffs.csv")

df.to_sql("LAYOFFS", conn, if_exists="append", chunksize=100, index=False, method="multi")

2361

# Data Cleaning

The first step is to make a staging table. This is where we'll work on cleaning the data. We need a table with the original data just in case something goes wrong.

In [ ]:
%%sql
DROP TABLE IF EXISTS LAYOFFS_STAGING;

CREATE TABLE LAYOFFS_STAGING
AS
SELECT *
FROM LAYOFFS

 * sqlite:///covid_data.db
Done.
Done.


[]

Let's check out the staging table.

In [ ]:
%%sql
PRAGMA TABLE_INFO(LAYOFFS_STAGING)

 * sqlite:///covid_data.db
Done.


cid,name,type,notnull,dflt_value,pk
0,company,TEXT,0,None,0
1,location,TEXT,0,None,0
2,industry,TEXT,0,None,0
3,total_laid_off,REAL,0,None,0
4,percentage_laid_off,REAL,0,None,0
5,date,TEXT,0,None,0
6,stage,TEXT,0,None,0
7,country,TEXT,0,None,0
8,funds_raised_millions,REAL,0,None,0


In [ ]:
%%sql
SELECT *
FROM LAYOFFS_STAGING
LIMIT 10

 * sqlite:///covid_data.db
Done.


company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
Atlassian,Sydney,Other,500.0,0.05,3/6/2023,Post-IPO,Australia,210.0
SiriusXM,New York City,Media,475.0,0.08,3/6/2023,Post-IPO,United States,525.0
Alerzo,Ibadan,Retail,400.0,None,3/6/2023,Series B,Nigeria,16.0
UpGrad,Mumbai,Education,120.0,None,3/6/2023,Unknown,India,631.0
Loft,Sao Paulo,Real Estate,340.0,0.15,3/3/2023,Unknown,Brazil,788.0
Embark Trucks,SF Bay Area,Transportation,230.0,0.7,3/3/2023,Post-IPO,United States,317.0
Lendi,Sydney,Real Estate,100.0,None,3/3/2023,Unknown,Australia,59.0
UserTesting,SF Bay Area,Marketing,63.0,None,3/3/2023,Acquired,United States,152.0
Airbnb,SF Bay Area,None,30.0,None,3/3/2023,Post-IPO,United States,6400.0
Accolade,Seattle,Healthcare,None,None,3/3/2023,Post-IPO,United States,458.0


Now, during data cleaning, we'll follow these steps:

1. Check for duplicates and eliminate them.
2. Standardize data and correct errors.
3. Examine null values and determine actions.
4. Remove any unnecessary columns and rows.

## 1. Remove Duplicates

First, we'll look for duplicates by using window functions and delete them from the staging table.

In [ ]:
%%sql
WITH duplicate_cte AS (
  SELECT
    ROW_NUMBER() OVER(PARTITION BY company, location, industry, total_laid_off, percentage_laid_off, date, stage, country, funds_raised_millions) AS rn,
    *
  FROM layoffs_staging
)

SELECT *
FROM duplicate_cte
WHERE rn > 1
ORDER BY company, rn

 * sqlite:///covid_data.db
Done.


rn,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
2,Casper,New York City,Retail,None,None,9/14/2021,Post-IPO,United States,339.0
2,Cazoo,London,Transportation,750.0,0.15,6/7/2022,Post-IPO,United Kingdom,2000.0
2,Hibob,Tel Aviv,HR,70.0,0.3,3/30/2020,Series A,Israel,45.0
2,Wildlife Studios,Sao Paulo,Consumer,300.0,0.2,11/28/2022,Unknown,Brazil,260.0
2,Yahoo,SF Bay Area,Consumer,1600.0,0.2,2/9/2023,Acquired,United States,6.0


Let's check the staging table to confirm the identified duplicates.

In [ ]:
%%sql
WITH duplicate_cte AS (
  SELECT
    ROW_NUMBER() OVER(PARTITION BY company, location, industry, total_laid_off, percentage_laid_off, date, stage, country, funds_raised_millions) AS rn,
    *
  FROM layoffs_staging
)

SELECT cte1.*
FROM duplicate_cte cte1
JOIN duplicate_cte cte2
  USING (company, location, industry, date)
WHERE cte2.rn > 1
ORDER BY company, rn

 * sqlite:///covid_data.db
Done.


rn,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1,Casper,New York City,Retail,None,None,9/14/2021,Post-IPO,United States,339.0
2,Casper,New York City,Retail,None,None,9/14/2021,Post-IPO,United States,339.0
1,Cazoo,London,Transportation,750.0,0.15,6/7/2022,Post-IPO,United Kingdom,2000.0
2,Cazoo,London,Transportation,750.0,0.15,6/7/2022,Post-IPO,United Kingdom,2000.0
1,Hibob,Tel Aviv,HR,70.0,0.3,3/30/2020,Series A,Israel,45.0
2,Hibob,Tel Aviv,HR,70.0,0.3,3/30/2020,Series A,Israel,45.0
1,Wildlife Studios,Sao Paulo,Consumer,300.0,0.2,11/28/2022,Unknown,Brazil,260.0
2,Wildlife Studios,Sao Paulo,Consumer,300.0,0.2,11/28/2022,Unknown,Brazil,260.0
1,Yahoo,SF Bay Area,Consumer,1600.0,0.2,2/9/2023,Acquired,United States,6.0
2,Yahoo,SF Bay Area,Consumer,1600.0,0.2,2/9/2023,Acquired,United States,6.0


These are exact duplicates, and we'll delete those where the row number is greater than 1.

One solution is to create a new column and add the row numbers into it. Then we can delete rows where the row numbers are greater than 1. So, let's proceed!

In [ ]:
%%sql
DROP TABLE IF EXISTS LAYOFFS_STAGING_V2;

CREATE TABLE LAYOFFS_STAGING_V2 (
  rn INT,
  company TEXT,
  location TEXT,
  industry TEXT,
  total_laid_off REAL,
  percentage_laid_off REAL,
  date TEXT,
  stage TEXT,
  country TEXT,
  funds_raised_millions REAL
);

INSERT INTO LAYOFFS_STAGING_V2
SELECT
  ROW_NUMBER() OVER(PARTITION BY company, location, industry, total_laid_off, percentage_laid_off, date, stage, country, funds_raised_millions) AS rn,
  *
FROM LAYOFFS_STAGING

 * sqlite:///covid_data.db
Done.
Done.
2361 rows affected.


[]

The staging table now includes a row number column.

In [ ]:
%%sql
SELECT *
FROM LAYOFFS_STAGING_V2
LIMIT 10

 * sqlite:///covid_data.db
Done.


rn,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1,E Inc.,Toronto,Transportation,None,None,12/16/2022,Post-IPO,Canada,None
1,Included Health,SF Bay Area,Healthcare,None,0.06,7/25/2022,Series E,United States,272.0
1,#Paid,Toronto,Marketing,19.0,0.17,1/27/2023,Series B,Canada,21.0
1,&Open,Dublin,Marketing,9.0,0.09,11/17/2022,Series A,Ireland,35.0
1,100 Thieves,Los Angeles,Consumer,12.0,None,7/13/2022,Series C,United States,120.0
1,100 Thieves,Los Angeles,Retail,None,None,1/10/2023,Series C,United States,120.0
1,10X Genomics,SF Bay Area,Healthcare,100.0,0.08,8/4/2022,Post-IPO,United States,242.0
1,1stdibs,New York City,Retail,70.0,0.17,4/2/2020,Series D,United States,253.0
1,2TM,Sao Paulo,Crypto,90.0,0.12,6/1/2022,Unknown,Brazil,250.0
1,2TM,Sao Paulo,Crypto,100.0,0.15,9/1/2022,Unknown,Brazil,250.0


Before we delete the duplicates, let's check the number of rows.

In [ ]:
%%sql
SELECT COUNT(*)
FROM LAYOFFS_STAGING_V2

 * sqlite:///covid_data.db
Done.


COUNT(*)
2361


Now, we'll delete rows where the row numbers are greater than 1.

In [ ]:
%%sql
DELETE FROM LAYOFFS_STAGING_V2 WHERE rn > 1

 * sqlite:///covid_data.db
5 rows affected.


[]

We've already removed 5 duplicate rows. Let's check the row count again.

In [ ]:
%%sql
SELECT COUNT(*)
FROM LAYOFFS_STAGING_V2

 * sqlite:///covid_data.db
Done.


COUNT(*)
2356


Everything looks good, let's proceed to the next step.

## 2. Standardize and Correct Errors

In this step, we'll review each column in the dataset to identify any errors or data that needs standardization. Let's begin with the 'company' column.

In [ ]:
%%sql
SELECT DISTINCT company
FROM LAYOFFS_STAGING_V2
ORDER BY company

 * sqlite:///covid_data.db
Done.


company
E Inc.
Included Health
#Paid
&Open
100 Thieves
10X Genomics
1stdibs
2TM
2U
54gene


It looks like some of the 'country' entries have redundant spaces, which can be identified with the following query.

In [ ]:
%%sql
SELECT DISTINCT company, TRIM(company)
FROM LAYOFFS_STAGING_V2
WHERE company != TRIM(company)
ORDER BY company

 * sqlite:///covid_data.db
Done.


company,TRIM(company)
E Inc.,E Inc.
Included Health,Included Health
Atlas Obscura,Atlas Obscura
Bonterra,Bonterra
Captain Fresh,Captain Fresh
Pear Therapeutics,Pear Therapeutics
Science 37,Science 37
Twine Solutions,Twine Solutions
WeWork,WeWork
Zymergen,Zymergen


Let's remove the redundant spaces and update those values in the 'country' column.

In [ ]:
%%sql
UPDATE LAYOFFS_STAGING_V2
SET company = TRIM(company)

 * sqlite:///covid_data.db
2356 rows affected.


[]

Let's check again.

In [ ]:
%%sql
SELECT COUNT(*)
FROM LAYOFFS_STAGING_V2
WHERE company != TRIM(company)

 * sqlite:///covid_data.db
Done.


COUNT(*)
0


Alright, next, let's examine the 'industry' column.

In [ ]:
%%sql
SELECT DISTINCT industry
FROM LAYOFFS_STAGING_V2
ORDER BY industry

 * sqlite:///covid_data.db
Done.


industry
None
Aerospace
Construction
Consumer
Crypto
Crypto Currency
CryptoCurrency
Data
Education
Energy


Looking at the industry column, it appears we have some null rows. We'll handle those in the next step. Additionally, we've observed that "Crypto" has multiple different variations.

In [ ]:
%%sql
SELECT DISTINCT industry
FROM LAYOFFS_STAGING_V2
WHERE industry LIKE 'Crypto%'
ORDER BY industry

 * sqlite:///covid_data.db
Done.


industry
Crypto
Crypto Currency
CryptoCurrency


We need to standardize that - let's change all variations to "Crypto".

In [ ]:
%%sql
UPDATE LAYOFFS_STAGING_V2
SET industry = 'Crypto'
WHERE industry LIKE 'Crypto%'

 * sqlite:///covid_data.db
102 rows affected.


[]

Now that's sorted out.

In [ ]:
%%sql
SELECT DISTINCT industry
FROM LAYOFFS_STAGING_V2
ORDER BY industry

 * sqlite:///covid_data.db
Done.


industry
None
Aerospace
Construction
Consumer
Crypto
Data
Education
Energy
Fin-Tech
Finance


Let's examine the 'location' column.

In [ ]:
%%sql
SELECT DISTINCT location
FROM LAYOFFS_STAGING_V2
ORDER BY location

 * sqlite:///covid_data.db
Done.


location
Accra
Ahmedabad
Albany
Amsterdam
Ann Arbor
Athens
Atlanta
Auckland
Austin
Baltimore


Looks good. We also need to review the country column.

In [ ]:
%%sql
SELECT DISTINCT country
FROM LAYOFFS_STAGING_V2
ORDER BY country

 * sqlite:///covid_data.db
Done.


country
Argentina
Australia
Austria
Bahrain
Belgium
Brazil
Bulgaria
Canada
Chile
China


Everything looks good except we have some entries as "United States" and some as "United States." with a period at the end.

In [ ]:
%%sql
SELECT DISTINCT country
FROM LAYOFFS_STAGING_V2
WHERE country LIKE 'United States%'
ORDER BY country

 * sqlite:///covid_data.db
Done.


country
United States
United States.


Let's standardize this.

In [ ]:
%%sql
SELECT DISTINCT country, TRIM(country, '.')
FROM LAYOFFS_STAGING_V2
WHERE country LIKE 'United States%'
ORDER BY country

 * sqlite:///covid_data.db
Done.


country,"TRIM(country, '.')"
United States,United States
United States.,United States


In [ ]:
%%sql
UPDATE LAYOFFS_STAGING_V2
SET country = TRIM(country, '.')
WHERE country LIKE 'United States%'

 * sqlite:///covid_data.db
1543 rows affected.


[]

Now, if we run this again, it should be fixed.

In [ ]:
%%sql
SELECT DISTINCT country
FROM LAYOFFS_STAGING_V2
WHERE country LIKE 'United States%'
ORDER BY country

 * sqlite:///covid_data.db
Done.


country
United States


Let's also correct the date columns.

In [ ]:
%%sql
PRAGMA TABLE_INFO(LAYOFFS_STAGING_V2)

 * sqlite:///covid_data.db
Done.


cid,name,type,notnull,dflt_value,pk
0,rn,INT,0,None,0
1,company,TEXT,0,None,0
2,location,TEXT,0,None,0
3,industry,TEXT,0,None,0
4,total_laid_off,REAL,0,None,0
5,percentage_laid_off,REAL,0,None,0
6,date,TEXT,0,None,0
7,stage,TEXT,0,None,0
8,country,TEXT,0,None,0
9,funds_raised_millions,REAL,0,None,0


We've noticed that the 'date' column is currently stored as a TEXT, which isn't the correct datatype for a date. To address this, we'll convert this column to a DATE datatype.

After that, we'll save the processed 'date' column along with the other columns we're interested in into another staging table for future analysis.

In [ ]:
%%sql
DROP TABLE IF EXISTS LAYOFFS_STAGING_V3;

CREATE TABLE LAYOFFS_STAGING_V3 (
  rn INT,
  company TEXT,
  location TEXT,
  industry TEXT,
  total_laid_off REAL,
  percentage_laid_off REAL,
  date DATE,
  stage TEXT,
  country TEXT,
  funds_raised_millions REAL
);

INSERT INTO LAYOFFS_STAGING_V3
WITH date_split AS (
  SELECT
    rn,
    company,
    location,
    industry,
    total_laid_off,
    percentage_laid_off,
    SUBSTR(date, 1, INSTR(date, '/') - 1) AS month,
    SUBSTR(SUBSTR(date, INSTR(date, '/') + 1), 1, INSTR(SUBSTR(date, INSTR(date, '/') + 1), '/') - 1) AS date,
    SUBSTR(SUBSTR(date, INSTR(date, '/') + 1), INSTR(SUBSTR(date, INSTR(date, '/') + 1), '/') + 1) AS year,
    stage,
    country,
    funds_raised_millions
  FROM LAYOFFS_STAGING_V2
),
date_normalize AS (
  SELECT
    rn,
    company,
    location,
    industry,
    total_laid_off,
    percentage_laid_off,
    year,
    CASE WHEN LENGTH(month) == 1 THEN '0' || month ELSE month END AS month,
    CASE WHEN LENGTH(date) == 1 THEN '0' || date ELSE date END AS date,
    stage,
    country,
    funds_raised_millions
  FROM date_split
)

SELECT
  rn,
  company,
  location,
  industry,
  total_laid_off,
  percentage_laid_off,
  DATE(year || '-' || month || '-' || date) AS date,
  stage,
  country,
  funds_raised_millions
FROM date_normalize

 * sqlite:///covid_data.db
Done.
Done.
2356 rows affected.


[]

Let's review our new staging table.

In [ ]:
%%sql
SELECT *
FROM LAYOFFS_STAGING_V3
ORDER BY company, location, industry, date
LIMIT 10

 * sqlite:///covid_data.db
Done.


rn,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1,#Paid,Toronto,Marketing,19.0,0.17,2023-01-27,Series B,Canada,21.0
1,&Open,Dublin,Marketing,9.0,0.09,2022-11-17,Series A,Ireland,35.0
1,100 Thieves,Los Angeles,Consumer,12.0,None,2022-07-13,Series C,United States,120.0
1,100 Thieves,Los Angeles,Retail,None,None,2023-01-10,Series C,United States,120.0
1,10X Genomics,SF Bay Area,Healthcare,100.0,0.08,2022-08-04,Post-IPO,United States,242.0
1,1stdibs,New York City,Retail,70.0,0.17,2020-04-02,Series D,United States,253.0
1,2TM,Sao Paulo,Crypto,90.0,0.12,2022-06-01,Unknown,Brazil,250.0
1,2TM,Sao Paulo,Crypto,100.0,0.15,2022-09-01,Unknown,Brazil,250.0
1,2U,Washington D.C.,Education,None,0.2,2022-07-28,Post-IPO,United States,426.0
1,54gene,Washington D.C.,Healthcare,95.0,0.3,2022-08-29,Series B,United States,44.0


## 3. Handle Null Values

Let's address the null values in the industry column as we've seen before.

In [ ]:
%%sql
SELECT *
FROM LAYOFFS_STAGING_V3
WHERE industry IS NULL
ORDER BY industry

 * sqlite:///covid_data.db
Done.


rn,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1,Airbnb,SF Bay Area,None,30.0,None,2023-03-03,Post-IPO,United States,6400.0
1,Bally's Interactive,Providence,None,None,0.15,2023-01-18,Post-IPO,United States,946.0
1,Carvana,Phoenix,None,2500.0,0.12,2022-05-10,Post-IPO,United States,1600.0
1,Juul,SF Bay Area,None,400.0,0.3,2022-11-10,Unknown,United States,1500.0


Now we need to fill in those nulls if possible. Let's examine some of these.

In [ ]:
%%sql
SELECT *
FROM LAYOFFS_STAGING_V3
WHERE company LIKE 'Bally%';

 * sqlite:///covid_data.db
Done.


rn,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1,Bally's Interactive,Providence,None,None,0.15,2023-01-18,Post-IPO,United States,946.0


Nothing seems to be amiss here.

In [ ]:
%%sql
SELECT *
FROM LAYOFFS_STAGING_V3
WHERE company LIKE 'airbnb%';

 * sqlite:///covid_data.db
Done.


rn,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1,Airbnb,SF Bay Area,None,30.0,None,2023-03-03,Post-IPO,United States,6400.0
1,Airbnb,SF Bay Area,Travel,1900.0,0.25,2020-05-05,Private Equity,United States,5400.0


It seems like "Airbnb" should be categorized as "Travel", but it's currently not populated.

I believe this might be the case for others as well. To address this, we can write a query that, for any row with a null industry but a company name that matches another row, it will update the industry to the non-null values.

This approach streamlines the process, avoiding manual checking for potentially thousands of entries.

In [ ]:
%%sql
UPDATE LAYOFFS_STAGING_V3
SET industry = t2.industry
FROM LAYOFFS_STAGING_V3 t2
WHERE LAYOFFS_STAGING_V3.company = t2.company
  AND LAYOFFS_STAGING_V3.industry IS NULL
  AND t2.industry IS NOT NULL

 * sqlite:///covid_data.db
3 rows affected.


[]

After checking, it seems that "Bally's" was the only one left without a populated row.

In [ ]:
%%sql
SELECT *
FROM LAYOFFS_STAGING_V3
WHERE industry IS NULL
ORDER BY industry

 * sqlite:///covid_data.db
Done.


rn,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1,Bally's Interactive,Providence,None,None,0.15,2023-01-18,Post-IPO,United States,946.0


Let's examine the null values in the 'total_laid_off', 'percentage_laid_off', and 'funds_raised_millions' columns.

In [ ]:
%%sql
SELECT *
FROM LAYOFFS_STAGING_V3
WHERE total_laid_off IS NULL
LIMIT 5

 * sqlite:///covid_data.db
Done.


rn,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1,E Inc.,Toronto,Transportation,None,None,2022-12-16,Post-IPO,Canada,None
1,Included Health,SF Bay Area,Healthcare,None,0.06,2022-07-25,Series E,United States,272.0
1,100 Thieves,Los Angeles,Retail,None,None,2023-01-10,Series C,United States,120.0
1,2U,Washington D.C.,Education,None,0.2,2022-07-28,Post-IPO,United States,426.0
1,5B Solar,Sydney,Energy,None,0.25,2022-06-03,Series A,Australia,12.0
1,80 Acres Farms,Cincinnati,Food,None,0.1,2023-01-18,Unknown,United States,275.0
1,98point6,Seattle,Healthcare,None,0.1,2022-07-21,Series E,United States,247.0
1,Accolade,Seattle,Healthcare,None,None,2023-03-03,Post-IPO,United States,458.0
1,ActiveCampaign,Chicago,Marketing,None,0.15,2022-10-03,Series C,United States,360.0
1,Ada,Toronto,Support,None,None,2023-02-01,Series C,Canada,190.0


In [ ]:
%%sql
SELECT *
FROM LAYOFFS_STAGING_V3
WHERE percentage_laid_off IS NULL
LIMIT 5

 * sqlite:///covid_data.db
Done.


rn,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1,E Inc.,Toronto,Transportation,None,None,2022-12-16,Post-IPO,Canada,None
1,100 Thieves,Los Angeles,Consumer,12.0,None,2022-07-13,Series C,United States,120.0
1,100 Thieves,Los Angeles,Retail,None,None,2023-01-10,Series C,United States,120.0
1,Absci,Vancouver,Healthcare,40.0,None,2022-08-09,Post-IPO,United States,237.0
1,Accolade,Seattle,Healthcare,None,None,2023-03-03,Post-IPO,United States,458.0
1,Acorns,Portland,Finance,50.0,None,2020-05-26,Unknown,United States,207.0
1,Actifio,Boston,Data,54.0,None,2020-12-16,Acquired,United States,352.0
1,Ada,Toronto,Support,None,None,2023-02-01,Series C,Canada,190.0
1,Ada Health,Berlin,Healthcare,50.0,None,2022-10-17,Series B,Germany,189.0
1,Adara,SF Bay Area,Travel,None,None,2020-03-31,Series C,United States,67.0


In [ ]:
%%sql
SELECT *
FROM LAYOFFS_STAGING_V3
WHERE funds_raised_millions IS NULL
LIMIT 5

 * sqlite:///covid_data.db
Done.


rn,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1,E Inc.,Toronto,Transportation,None,None,2022-12-16,Post-IPO,Canada,None
1,Advata,Seattle,Healthcare,32.0,0.21,2022-10-28,None,United States,None
1,Afterverse,Brasilia,Consumer,60.0,0.2,2022-06-03,Unknown,Brazil,None
1,Agoda,Singapore,Travel,1500.0,0.25,2020-05-18,Acquired,Singapore,None
1,Airy Rooms,Jakarta,Travel,None,0.7,2020-04-22,Unknown,Indonesia,None
1,Airy Rooms,Jakarta,Travel,None,1.0,2020-05-07,Unknown,Indonesia,None
1,Akerna,Denver,Logistics,None,None,2020-09-02,Post-IPO,United States,None
1,Akili Labs,Baltimore,Healthcare,46.0,0.3,2023-01-12,Seed,United States,None
1,Ambev Tech,Blumenau,Food,50.0,None,2023-03-02,Acquired,Brazil,None
1,Amdocs,St. Louis,Support,700.0,0.03,2023-01-02,Post-IPO,United States,None


The null values in 'total_laid_off', 'percentage_laid_off', and 'funds_raised_millions' all seem fine. We prefer leaving them as null because it simplifies calculations during the Exploratory Data Analysis (EDA) phase.

So, there's nothing we want to alter with the null values in those columns.

## 4. Trim Unnecessary Columns and Rows

Let's delete the data that we can't really use.

In [ ]:
%%sql
SELECT *
FROM LAYOFFS_STAGING_V3
WHERE total_laid_off IS NULL
  AND percentage_laid_off IS NULL
LIMIT 10

 * sqlite:///covid_data.db
Done.


rn,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
1,E Inc.,Toronto,Transportation,None,None,2022-12-16,Post-IPO,Canada,None
1,100 Thieves,Los Angeles,Retail,None,None,2023-01-10,Series C,United States,120.0
1,Accolade,Seattle,Healthcare,None,None,2023-03-03,Post-IPO,United States,458.0
1,Ada,Toronto,Support,None,None,2023-02-01,Series C,Canada,190.0
1,Adara,SF Bay Area,Travel,None,None,2020-03-31,Series C,United States,67.0
1,Addi,Bogota,Finance,None,None,2022-06-14,Series C,Colombia,376.0
1,AirMap,Los Angeles,Aerospace,None,None,2020-04-30,Unknown,United States,75.0
1,Airtasker,Sydney,Consumer,None,None,2022-07-04,Series C,Australia,26.0
1,Akerna,Denver,Logistics,None,None,2022-05-27,Unknown,United States,46.0
1,Akerna,Denver,Logistics,None,None,2020-09-02,Post-IPO,United States,None


In [ ]:
%%sql
DELETE FROM LAYOFFS_STAGING_V3
WHERE total_laid_off IS NULL
  AND percentage_laid_off IS NULL

 * sqlite:///covid_data.db
361 rows affected.


[]

Upon rechecking, we can see that there are no longer any rows with null values in total_laid_off and percentage_laid_off.

In [ ]:
%%sql
SELECT COUNT(*)
FROM LAYOFFS_STAGING_V3
WHERE total_laid_off IS NULL
  AND percentage_laid_off IS NULL

 * sqlite:///covid_data.db
Done.


COUNT(*)
0


Lastly, we'll drop the 'rn' column.

In [ ]:
%%sql
ALTER TABLE LAYOFFS_STAGING_V3
DROP COLUMN rn

 * sqlite:///covid_data.db
Done.


[]

Let's review our final dataset.

In [ ]:
%%sql
SELECT *
FROM LAYOFFS_STAGING_V3
LIMIT 10

 * sqlite:///covid_data.db
Done.


company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
Included Health,SF Bay Area,Healthcare,None,0.06,2022-07-25,Series E,United States,272.0
#Paid,Toronto,Marketing,19.0,0.17,2023-01-27,Series B,Canada,21.0
&Open,Dublin,Marketing,9.0,0.09,2022-11-17,Series A,Ireland,35.0
100 Thieves,Los Angeles,Consumer,12.0,None,2022-07-13,Series C,United States,120.0
10X Genomics,SF Bay Area,Healthcare,100.0,0.08,2022-08-04,Post-IPO,United States,242.0
1stdibs,New York City,Retail,70.0,0.17,2020-04-02,Series D,United States,253.0
2TM,Sao Paulo,Crypto,90.0,0.12,2022-06-01,Unknown,Brazil,250.0
2TM,Sao Paulo,Crypto,100.0,0.15,2022-09-01,Unknown,Brazil,250.0
2U,Washington D.C.,Education,None,0.2,2022-07-28,Post-IPO,United States,426.0
54gene,Washington D.C.,Healthcare,95.0,0.3,2022-08-29,Series B,United States,44.0


# Save the Result

After completing the cleaning steps, we should save the processed table into a CSV file for future use.

In [ ]:
# Query to select all records from the table
query = 'SELECT * FROM LAYOFFS_STAGING_V3'

# Load all the records into a dataframe.
df = pd.read_sql(query, conn, index_col=None)

# Print some first rows of the dataframe.
df.head()

,company,location,industry,total_laid_off,percentage_laid_off,date,stage,country,funds_raised_millions
0,Included Health,SF Bay Area,Healthcare,NaN,0.06,2022-07-25,Series E,United States,272.0
1,#Paid,Toronto,Marketing,19.0,0.17,2023-01-27,Series B,Canada,21.0
2,&Open,Dublin,Marketing,9.0,0.09,2022-11-17,Series A,Ireland,35.0
3,100 Thieves,Los Angeles,Consumer,12.0,NaN,2022-07-13,Series C,United States,120.0
4,10X Genomics,SF Bay Area,Healthcare,100.0,0.08,2022-08-04,Post-IPO,United States,242.0


In [ ]:
# Write the dataframe to a csv file
df.to_csv('layoffs_staging.csv')

# Conclusion

This project successfully cleaned and refined the "Layoffs 2022" dataset from Kaggle, preparing it for in-depth analysis. Key accomplishments include:

- Removed duplicates and standardized data for accuracy.
- Corrected date formats and retained relevant NULL values.
- Eliminated unnecessary data for clarity.

## Next Steps
With a clean dataset, Part 2 will focus on Exploratory Data Analysis (EDA) to uncover insights. Future analysis could include:

- Understanding industry and location trends in layoffs.
- Analyzing the impact of funding stages on layoff numbers.
- Investigating correlations between layoffs and funds raised.